In [ ]:
import pandas as pd

In [ ]:
#preparing and cleaning data:
df = pd.read_csv('./assets/data/UFO_raw.csv')

In [ ]:
# dropping unnecessary columns:
try:
    df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    pass

# capitalizing each word and removing redundant spaces:
df = df.apply(lambda x: x.str.title() if x.dtype == object else x)


# replacing unnecessary strings:
df.replace(['\(Poland\)', 'Voivodeship', 'Województwo'], '', regex=True, inplace=True)
df.replace('Undisclosed', 'No data', inplace=True)
df.fillna('No data', inplace=True)

# splitting column into two columns:
df[['Town', 'Voivodeship']] = df['Location'].str.split(',', n=1, expand=True)

# dropping original column:
try:
    df.drop('Location', axis=1, inplace=True)
except:
    pass

# rearranging columns' order:
df = df[['Town', 'Voivodeship', 'Sighted on', 'Shape', 'Duration']]

# removing redundant spaces:
df = df.apply(lambda x: x.str.strip() if x.dtype == object else x)

In [ ]:
voivodeships_dict = {
    'Zachodniopomorskie': 'West Pomeranian',
    'Pomorskie': 'Pomeranian',
    'Warmińsko-Mazurskie': 'Warmian-Masurian',
    'Podlaskie': 'Podlaskie',
    'Lubuskie': 'Lubusz',
    'Wielkopolskie': 'Greater Poland',
    'Kujawsko-Pomorskie': 'Kujavian-Pomeranian',
    'Dolnośląskie': 'Lower Silesian',
    'Opolskie': 'Opole',
    'Śląskie': 'Silesian',
    'Świętokrzyskie': 'Swietokrzyskie',
    'Łódzkie': 'Lodz',
    'Mazowieckie': 'Masovian',
    'Lubelskie': 'Lublin',
    'Małopolskie': 'Lesser Poland',
    'Podkarpackie': 'Subcarpathian'
}

polish_dict = {
    'ą': 'a', 
    'ć': 'c', 
    'ę': 'e',
    'ł': 'l',
    'ń': 'n', 
    'ó': 'o', 
    'ś': 's',
    'ź': 'z', 
    'ż': 'z'
}

In [ ]:
# replacing Polish voivodeships' names and Polish letters by their English counterparts:
df.replace(voivodeships_dict.keys(), voivodeships_dict.values(), regex=True, inplace=True)
df['Town'] = df['Town'].str.lower()
df.replace(polish_dict.keys(), polish_dict.values(), regex=True, inplace=True)
df['Town'] = df['Town'].str.title()
df['Year'] = df['Sighted on'].str.split('-', n=1, expand=True)[0]

In [ ]:
df.to_csv('./assets/data/UFO_clean.csv', index=False)

In [ ]:
# preparing coordinates dataset:
df = pd.read_csv('./assets/data/Coordinates.csv', sep='\tab', engine='python')

df[['Town1', 'Town2', 'Coordinates']] = df.iloc[:, 0].str.split(' ', n=2, expand=True)
df.replace(['°', '\'E', '\'N'], ['.', ', ', ''], regex=True, inplace=True)
df[['Long', 'Lat']] = df['Coordinates'].str.split(',', n=3, expand=True)
df['Town'] = df[['Town1', 'Town2']].agg(' '.join, axis=1)
df.drop(df.iloc[:, :2], axis=1, inplace=True)
df = df[['Town', 'Coordinates', 'Lat', 'Long']]
df['Town'] = df['Town'].str.lower()
df.replace(polish_dict.keys(), polish_dict.values(), regex=True, inplace=True)
df['Town'] = df['Town'].str.title()
df = df.apply(lambda x: x.str.strip() if x.dtype == object else x)

In [ ]:
df.to_csv('./assets/data/Coordinates_clean.csv', index=False)

In [ ]:
# merging two dataframes:
df_ufo = pd.read_csv('./assets/data/UFO_clean.csv')
df_coords = pd.read_csv('./assets/data/Coordinates_clean.csv')
df_merged = df_ufo.merge(df_coords[['Town', 'Lat', 'Long']], on='Town')

In [ ]:
df_merged.to_csv('./assets/data/Clean_data.csv')